Matrícula: 	564307\
Nível: 	MESTRADO\
Status: 	ATIVO\
E-Mail: 	danieloliveirafff@gmail.com\
Entrada: 	2024.1\
Orientador: 	JOAO PAULO DO VALE MADEIRO\
Área: 	CIÊNCIA DA COMPUTAÇÃO\
Linha de Pesquisa: 	TEORIA DA COMPUTAÇÃO

In [1]:
# Bibliotecas

# Tratamento de dados
import numpy as np
import pandas as pd


# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette("Accent")
sns.set_style("darkgrid")
import plotly.express as px
import plotly.graph_objects as g


# Transformação dos dados
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split


In [2]:
# carregando os dados
# motando o google drive

from google.colab import drive
drive.mount('/content/drive')

link = '/content/drive/MyDrive/UFC_mestrado/Sigaa_UFC/1_semestre/aprendizagem_automatica/lista_04_ama/dados/concrete.csv'

df = pd.read_csv(link, header=None)

dados = np.array(df)

Mounted at /content/drive


# **TRATAMENTO DE DADOS**

In [3]:
# separando os dados de x e de y
# X = dados[:, :-1]  # As 8 primeiras colunas são os atributos
# y = dados[:, -1]  # A última coluna é a saída
# y = np.array(y).reshape(-1, 1)

np.random.seed(420) # para manter aleatoriedade

# Separar os atributos (X) e a saída (y)
X = dados[:, :-1]  # As 8 primeiras colunas são os atributos
y = dados[:, -1]   # A última coluna é a saída
y = np.array(y).reshape(-1, 1)


In [4]:
# Gerar um array de índices embaralhados
indices = np.random.permutation(len(X))

# Usar esses índices para embaralhar X e y de forma consistente
X_embaralhado = X[indices]
y_embaralhado = y[indices]

In [5]:
# normalizando os dados com minimo e e max
# from sklearn.preprocessing import MinMaxScaler

# # Normalizando os dados de entrada
# scaler_X = MinMaxScaler()
# X_normalizado = scaler_X.fit_transform(X)

# # Normalizando os dados de saída
# scaler_y = MinMaxScaler()
# y_normalizado = scaler_y.fit_transform(y.reshape(-1, 1))  # y_1 precisa ser 2D para o MinMaxScaler


In [6]:
# # normalizando os dados em binarios
# from sklearn.preprocessing import Binarizer

# # Binarizando os dados de entrada
# binarizador_X = Binarizer()
# X_binarizado = binarizador_X.fit_transform(X)

# Binarizar y (supondo que você tenha uma classificação binária)
# binarizer = LabelBinarizer()
# y_binarizado = binarizer.fit_transform(y)


In [7]:

# Dividir os dados em treino (60%), validação (20%) e teste (20%)
# X_train, X_temp, y_train, y_temp = train_test_split(X_normalizado, y_normalizado, test_size=0.4, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
# Normalizando o X e o Y com min-max

# normalizacao = lambda x : (x - np.mean(x, axis = 0)) / np.std(x, axis = 0)
normalizacao = lambda x : (x - np.min(x, axis=0)) / (np.max(x, axis=0) - np.min(x, axis=0))

# normalizando o x
x_normalizado = normalizacao(X_embaralhado)
y_normalizado = normalizacao(y_embaralhado)
x_normalizado.shape

(1030, 8)

In [9]:
# limitando a quantidade de linha para treino, teste e validação do x_normalizado e y_normalizado
# Dividir os dados em treino (60%), validação (20%) e teste (20%)

tamanho_treino = int(len(x_normalizado) * 0.6)  # Treino (60%)
tamanho_validacao = int(len(x_normalizado) * 0.2)  # Validação (20%)
tamanho_teste = len(x_normalizado) - tamanho_treino - tamanho_validacao  # Teste (20%)

# Dividir os dados em treino, validação e teste
xtreino = x_normalizado[:tamanho_treino]
ytreino = y_normalizado[:tamanho_treino]

xvalidacao = x_normalizado[tamanho_treino:tamanho_treino + tamanho_validacao]
yvalidacao = y_normalizado[tamanho_treino:tamanho_treino + tamanho_validacao]

xteste = x_normalizado[tamanho_treino + tamanho_validacao:]
yteste = y_normalizado[tamanho_treino + tamanho_validacao:]


# **TREINANDO O MODELO REDE NEURAIS**

O objetivo dessa qustão é criar e implementar Redes Neurais de acordo com os  requisito da Questão.  
será dividido em três partes importante para estrutura do modelo dos quais são:
- Class do modelo de rede neurais
- Class de métricas
- Class dos gráficos





In [19]:
import numpy as np

class RedesNeurais:
    def __init__(self):
        self.pesos = []
        self.biases = []
        self.ativacoes = []
        self.ativacoes_deriv = []
        self.momentum_pesos = [] # vetores para o método Adam
        self.momentum_biases = [] # vetorias para o método Adam
        self.v_pesos = []
        self.v_biases = []
        self.t = 0  # Para contar as iterações no Adam
        self.y_train = None  # recebe o y-treino
        self.y_test = None # recebe o y-teste
        self.y_previsto_train = None  # y calculado
        self.y_previsto_val = None
        self.y_previsto_test = None
        self.loss = []  # Para armazenar as perdas
        self.val_loss = []  # Para armazenar as perdas no conjunto de validação

    def adicionar_camada(self, n_entrada, n_saida, ativacao='relu'):
      #inicialização dos pesos e bias
        self.pesos.append(np.random.randn(n_entrada, n_saida) * np.sqrt(2. / n_entrada))
        self.biases.append(np.zeros((1, n_saida)))

        # Inicializar momentos e v para Adam
        self.momentum_pesos.append(np.zeros_like(self.pesos[-1]))
        self.momentum_biases.append(np.zeros_like(self.biases[-1]))
        self.v_pesos.append(np.zeros_like(self.pesos[-1]))
        self.v_biases.append(np.zeros_like(self.biases[-1]))

        if ativacao == 'relu':
            self.ativacoes.append(self.relu)
            self.ativacoes_deriv.append(self.relu_derivative)
        elif ativacao == 'leaky_relu':
            self.ativacoes.append(self.leaky_relu)
            self.ativacoes_deriv.append(self.leaky_relu_derivative)
        elif ativacao == 'linear':
            self.ativacoes.append(self.linear)
            self.ativacoes_deriv.append(self.linear_derivative)
        else:
            raise ValueError(f"Função de ativação {ativacao} não suportada.")

    def adam_optimizer(self, grad, m, v, beta1, beta2, epsilon, learning_rate):
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * (grad ** 2)
        m_corrigido = m / (1 - beta1 ** (self.t + 1))
        v_corrigido = v / (1 - beta2 ** (self.t + 1))
        return m, v, -learning_rate * m_corrigido / (np.sqrt(v_corrigido) + epsilon)

    def forward(self, X):
        self.entradas = [X]
        self.saidas = []

        for i in range(len(self.pesos)):
            z = np.dot(self.entradas[-1], self.pesos[i]) + self.biases[i]
            a = self.ativacoes[i](z)
            self.entradas.append(z)
            self.saidas.append(a)

        return self.saidas[-1]

    def calcular_perda(self, X, y, learning_rate, regularization_lambda=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        # Gradiente da camada de saída
        erro_saida = self.saidas[-1] - y
        grad_saida = erro_saida * self.ativacoes_deriv[-1](self.saidas[-1])

        self.t += 1  # Incrementar o contador de iterações

        # Atualização da camada de saída com Adam e regularização
        self.momentum_pesos[-1], self.v_pesos[-1], delta_pesos = self.adam_optimizer(
            np.dot(self.entradas[-2].T, grad_saida) + regularization_lambda * self.pesos[-1],
            self.momentum_pesos[-1],
            self.v_pesos[-1],
            beta1, beta2, epsilon,
            learning_rate
        )
        self.pesos[-1] += delta_pesos

        self.momentum_biases[-1], self.v_biases[-1], delta_biases = self.adam_optimizer(
            np.sum(grad_saida, axis=0, keepdims=True),
            self.momentum_biases[-1],
            self.v_biases[-1],
            beta1, beta2, epsilon,
            learning_rate
        )
        self.biases[-1] += delta_biases

        erro_propagado = grad_saida

        # Retropropagação nas camadas ocultas com Adam e regularização
        for i in reversed(range(len(self.pesos) - 1)):
            grad_ativacao = self.ativacoes_deriv[i](self.saidas[i])
            erro_propagado = np.dot(erro_propagado, self.pesos[i + 1].T) * grad_ativacao

            self.momentum_pesos[i], self.v_pesos[i], delta_pesos = self.adam_optimizer(
                np.dot(self.entradas[i].T, erro_propagado) + regularization_lambda * self.pesos[i],
                self.momentum_pesos[i],
                self.v_pesos[i],
                beta1, beta2, epsilon,
                learning_rate
            )
            self.pesos[i] += delta_pesos

            self.momentum_biases[i], self.v_biases[i], delta_biases = self.adam_optimizer(
                np.sum(erro_propagado, axis=0, keepdims=True),
                self.momentum_biases[i],
                self.v_biases[i],
                beta1, beta2, epsilon,
                learning_rate
            )
            self.biases[i] += delta_biases

    def treinar(self, X_train, y_train, X_val, y_val, epochs, learning_rate):
        self.loss = []  # Inicializa a lista de perdas
        self.val_loss = []  # Inicializa a lista de perdas de validação
        self.y_val = y_val
        self.y_train = y_train

        for epoch in range(epochs):
            self.y_previsto_train = self.forward(X_train)
            self.calcular_perda(X_train, self.y_train, learning_rate)
            train_loss = np.mean((self.y_train - self.y_previsto_train) ** 2)
            self.loss.append(train_loss)

            # Avaliação no conjunto de validação
            self.y_previsto_val = self.forward(X_val)
            val_loss = np.mean((y_val - self.y_previsto_val) ** 2)
            self.val_loss.append(val_loss)

            if epoch % 50 == 0:
                print(f'Epoch {epoch}, Train Loss: {train_loss}, Validation Loss: {val_loss}')

    def prever(self, X):
        return self.forward(X)


    @staticmethod
    def linear(x):
        return x

    @staticmethod
    def relu(x):
        return np.maximum(0, x)

    @staticmethod
    def leaky_relu(x, alpha=0.01):
        return np.where(x > 0, x, alpha * x)

    @staticmethod
    def linear_derivative(x):
        return np.ones_like(x)

    @staticmethod
    def relu_derivative(x):
        return np.where(x > 0, 1, 0)

    @staticmethod
    def leaky_relu_derivative(x, alpha=0.01):
        return np.where(x > 0, 1, alpha)


  #__________________________Métricas__gráficos____________________________________
    def metricas(self, para, tipo):
        if para == 'treino':
            y_true =  self.y_train
            y_pred =  self.y_previsto_train
        elif para == 'validacao':
            y_true =  self.y_val
            y_pred =  self.y_previsto_val
        elif para == 'teste':
            y_true =  self.y_test
            y_pred =  self.y_previsto_test
        else:
            raise ValueError("O parâmetro 'para' deve ser 'treino', 'validacao' ou 'teste'")

        if tipo == 'MAE':
            return np.mean(np.abs(y_true - y_pred))
        elif tipo == 'MSE':
            return np.mean((y_true - y_pred) ** 2)
        elif tipo == 'RMSE':
            return np.sqrt(np.mean((y_true - y_pred) ** 2))
        elif tipo == 'MRE':
            return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100
        elif tipo == 'MAPE':
            return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100
        elif tipo == "regressao":
            tabela = pd.DataFrame({
                'Erro Médio Absoluto (MAE)': [np.mean(np.abs(y_true - y_pred))],
                'Erro Quadrático Médio (MSE)': [np.mean((y_true - y_pred) ** 2)],
                'Raiz do Erro Quadrático Médio (RMSE)': [np.sqrt(np.mean((y_true - y_pred) ** 2))],
                #'Erro Relativo Médio (MRE)': [np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100]
            })
            tabela_estilo = tabela.style.format('{:.4f}')
            tabela_estilo.set_table_styles([
                {'selector': 'th', 'props': 'font-family: Helvetica; color: #dddd55; background-color: #34495E; text-align: center;'},
                {'selector': 'td', 'props': 'font-family: Helvetica; color: white; background-color: #34495E; text-align: left;'}
            ])
            return tabela_estilo
        else:
            raise ValueError(f"Métrica {tipo} não suportada.")

    # def metricas(self,para,tipo):

    #     # Reverter normalização
    #     max_y = np.max(self.y_train)
    #     min_y = np.min(self.y_train)


    #     def retorna_metricas(y_true, y_pred, tipo):
    #         if y_true is None or y_pred is None:
    #             raise ValueError("Você precisa treinar a rede antes de calcular as métricas.")

    #         # Verificar se os dados são unidimensionais ou não
    #         if y_true.ndim > 1:
    #             y_true = np.argmax(y_true, axis=1)
    #         if y_pred.ndim > 1:
    #             y_pred = np.argmax(y_pred, axis=1)

    #         # Reverter normalização
    #         #y_true = y_true * (max_y - min_y) + min_y
    #         #y_pred = y_pred * (max_y - min_y) + min_y

    #         # Valor absoluto
    #         erros_absolutos = np.abs(y_true - y_pred)
    #         total = len(y_true)
    #         regul = 1e-10  # Para evitar divisão por zero

    #         if tipo == 'MAE':
    #             return np.sum(erros_absolutos) / total
    #         elif tipo == 'MSE':
    #             return np.sum(np.power(y_true - y_pred, 2)) / total
    #         elif tipo == 'RMSE':
    #             return np.sqrt(np.sum(np.power(y_true - y_pred, 2)) / total)
    #         elif tipo == 'MRE':
    #             return np.mean(np.abs((y_true - y_pred) / (y_true + regul))) * 100
    #         elif tipo == 'MAPE':
    #             erro_percentual = np.abs(y_true - y_pred) / (y_true + regul)
    #             return np.mean(erro_percentual) * 100
    #         elif tipo == "regressao":
    #             tabela = pd.DataFrame({
    #                 'Erro Médio Absoluto (MAE)': [np.sum(erros_absolutos) / total],
    #                 'Erro Quadrático Médio (MSE)': [np.sum(np.power(y_true - y_pred, 2)) / total],
    #                 'Raiz do Erro Quadrático Médio (RMSE)': [np.sqrt(np.sum(np.power(y_true - y_pred, 2)) / total)],
    #                 'Erro Relativo Médio (MRE)': [np.mean(np.abs((y_true - y_pred) / (y_true + regul))) * 100]
    #             })
    #             tabela_estilo = tabela.style.format('{:.2f}')
    #             tabela_estilo.set_table_styles([
    #                 {'selector': 'th', 'props': 'font-family: Helvetica; color: #dddd55; background-color: #34495E; text-align: center;'},
    #                 {'selector': 'td', 'props': 'font-family: Helvetica; color: white; background-color: #34495E; text-align: left;'}
    #             ])
    #             return tabela_estilo
    #         else:
    #             raise ValueError(f"Métrica {tipo} não suportada.")


    #     if para == 'treino':
    #         y_true = np.argmax(self.y_train, axis=1) if self.y_train.ndim > 1 else self.y_train
    #         y_pred = np.argmax(self.y_previsto_train, axis=1) if self.y_previsto_train.ndim > 1 else self.y_previsto_train
    #         return retorna_metricas(y_true, y_pred, tipo)
    #     elif para == 'validacao':
    #         y_true = np.argmax(self.y_val, axis=1) if self.y_val.ndim > 1 else self.y_val
    #         y_pred = np.argmax(self.y_previsto_val, axis=1) if self.y_previsto_val.ndim > 1 else self.y_previsto_val
    #         return retorna_metricas(y_true, y_pred, tipo)
    #     elif para == 'teste':
    #         y_true = np.argmax(self.y_test, axis=1) if self.y_test.ndim > 1 else self.y_test
    #         y_pred = np.argmax(self.y_previsto_test, axis=1) if self.y_previsto_test.ndim > 1 else self.y_previsto_test
    #         return retorna_metricas(y_true, y_pred, tipo)

    def depurar_dados(self):
        print("Valores Reais (Treinamento):", self.y_train)
        print("Previsões (Treinamento):", self.y_previsto_train)
        print("Diferença (Treinamento):", self.y_train - self.y_previsto_train)

        print("Valores Reais (Validação):", self.y_val)
        print("Previsões (Validação):", self.y_previsto_val)
        print("Diferença (Validação):", self.y_val - self.y_previsto_val)

        print("Valores Reais (Teste):", self.y_test)
        print("Previsões (Teste):", self.y_previsto_test)
        print("Diferença (Teste):", self.y_test - self.y_previsto_test)



    #________________________gráficos________________________________
    def grafico_perda(self):
          if not hasattr(self, 'loss') or self.loss is None:
              raise ValueError("A lista de perdas 'self.loss' não está definida.")

          if isinstance(self.loss, np.ndarray):
              self.loss = self.loss.tolist()  # Converte numpy array para lista, se necessário

          if isinstance(self.val_loss, np.ndarray):
              self.val_loss = self.val_loss.tolist()  # Converte numpy array para lista, se necessário


          # Criar uma figura vazia
          fig = px.line(title='Gráfico Loss', labels={'x': 'Épocas', 'y': 'Loss'}, line_shape='linear')

          # Adicionar a linha de treino com a legenda "Treino"
          fig.add_scatter(x=list(range(len(self.loss))), y=self.loss, mode='lines', name='Treino', line=dict(width=6))

          # Adicionar a linha de validação com a legenda "Validação"
          fig.add_scatter(x=list(range(len(self.val_loss))), y=self.val_loss, mode='lines', name='Validação', line=dict(width=4))


          fig.update_layout(
              xaxis_title='Épocas',
              yaxis_title='Loss',
              hovermode='x',
              template='plotly_white',
              width=1400,
              height=600,
              margin=dict(l=20, r=20, t=35, b=20),
              title_font=dict(size=20, color='black'),
              xaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black')),
              yaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black'))
          )

          return fig.show()

    def histograma_frequencia(self):

          data = {
              'Valores':np.concatenate((self.y_train, self.y_previsto_train)),
              'Tipo': ['Y'] * len(self.y_train) + ['PREVISTO'] * len(self.y_previsto_train)
              }

          df = pd.DataFrame(data)

          # Criar um histograma para visualizar a frequência dos valores
          fig = px.histogram(df, x='Valores', color='Tipo', barmode='overlay', title='Histograma de Frequências de y_treino e y_previsto_train')

          fig.update_layout(
              xaxis_title='Valores',
              yaxis_title='Frequência',
              template='plotly_white',
              width=1200,
              height=400,
              title_font=dict(size=20, color='black'),
              xaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black')),
              yaxis=dict(title=dict(font=dict(size=14, color='black')), tickfont=dict(color='black'))
          )

          return fig.show()


In [11]:
# rede_neural = RedesNeurais()

# n_features = 8  # Número de características de entrada

# # Adicionando camadas: entrada (8 neurônios), ocultas (4 neurônios), e saída (1 neurônio)
# rede_neural.adicionar_camada(n_features, 8, ativacao='relu')  # Primeira camada oculta
# rede_neural.adicionar_camada(8, 8, ativacao='relu')  # Segunda camada oculta
# rede_neural.adicionar_camada(8, 1, ativacao='linear')  # Camada de saída

# Treinando a rede
# rede_neural.treinar(X_train, y_train,X_val,y_val, epochs=5000, learning_rate=0.001)

# # Fazendo previsões
# previsoes = rede_neural.prever(X_normalizado)
# print("Previsões:", previsoes)

In [20]:
rede_neural = RedesNeurais()

n_features = 8  # Número de características de entrada

# Adicionando camadas: entrada (8 neurônios), ocultas (4 neurônios), e saída (1 neurônio)
rede_neural.adicionar_camada(n_features, 8, ativacao='relu')  # Primeira camada oculta
rede_neural.adicionar_camada(8, 8, ativacao='relu')  # Segunda camada oculta
rede_neural.adicionar_camada(8, 1, ativacao='linear')  # Camada de saída

# Treinando a rede
rede_neural.treinar(xtreino, ytreino,xvalidacao, yvalidacao, epochs=5000, learning_rate=0.001)

# # Fazendo previsões
# previsoes = rede_neural.prever(X_normalizado)
# print("Previsões:", previsoes)

Epoch 0, Train Loss: 0.2148267694433263, Validation Loss: 0.20161407866156017
Epoch 50, Train Loss: 0.04982657220283817, Validation Loss: 0.05293573331189012
Epoch 100, Train Loss: 0.031085315471863655, Validation Loss: 0.030766994012979238
Epoch 150, Train Loss: 0.025272937395606196, Validation Loss: 0.024207480860578882
Epoch 200, Train Loss: 0.022657752262729727, Validation Loss: 0.022042228984130132
Epoch 250, Train Loss: 0.02112947807893228, Validation Loss: 0.020844591702786505
Epoch 300, Train Loss: 0.020109397746158802, Validation Loss: 0.020043001000155214
Epoch 350, Train Loss: 0.01932099111704904, Validation Loss: 0.01952243779457229
Epoch 400, Train Loss: 0.018677064824836956, Validation Loss: 0.01915770783708928
Epoch 450, Train Loss: 0.018191664147956595, Validation Loss: 0.01878928293455983
Epoch 500, Train Loss: 0.01783919161944733, Validation Loss: 0.01858557440478208
Epoch 550, Train Loss: 0.017599701524577387, Validation Loss: 0.018469564073606382
Epoch 600, Train Lo

In [21]:
rede_neural.grafico_perda()

In [22]:
# Fazer previsões
y_train_previsto = rede_neural.prever(xtreino)
y_val_previsto = rede_neural.prever(xvalidacao)
y_test_previsto = rede_neural.prever(xteste)

# Calcular e imprimir métricas para o conjunto de treinamento
rede_neural.y_train = ytreino  # Certifique-se de que y_train é o valor correto
rede_neural.y_previsto_train = y_train_previsto  # Atribua as previsões para o treinamento
print("Treinamento:")
print("MAE:", rede_neural.metricas('treino', 'MAE'))
print("MSE:", rede_neural.metricas('treino', 'MSE'))
print("RMSE:", rede_neural.metricas('treino', 'RMSE'))

# Avaliação no conjunto de validação
rede_neural.y_val = yvalidacao  # Atribua os valores reais para validação
rede_neural.y_previsto_val = y_val_previsto  # Atribua as previsões para validação
print("\nValidação:")
print("MAE:", rede_neural.metricas('validacao', 'MAE'))
print("MSE:", rede_neural.metricas('validacao', 'MSE'))
print("RMSE:", rede_neural.metricas('validacao', 'RMSE'))

# Avaliação no conjunto de teste
rede_neural.y_test = yteste  # Atribua os valores reais para teste
rede_neural.y_previsto_test = y_test_previsto  # Atribua as previsões para teste
print("\nTeste:")
print("MAE:", rede_neural.metricas('teste', 'MAE'))
print("MSE:", rede_neural.metricas('teste', 'MSE'))
print("RMSE:", rede_neural.metricas('teste', 'RMSE'))


Treinamento:
MAE: 0.10516711415165629
MSE: 0.017437861994920346
RMSE: 0.13205249711732203

Validação:
MAE: 0.10843640070298995
MSE: 0.01960922764618235
RMSE: 0.14003295200124274

Teste:
MAE: 0.09458935560451186
MSE: 0.014096392315565417
RMSE: 0.11872822880665498


In [23]:
rede_neural.metricas('treino','regressao')

,Erro Médio Absoluto (MAE),Erro Quadrático Médio (MSE),Raiz do Erro Quadrático Médio (RMSE)
0,0.1052,0.0174,0.1321


In [24]:
rede_neural.metricas('validacao','regressao')

,Erro Médio Absoluto (MAE),Erro Quadrático Médio (MSE),Raiz do Erro Quadrático Médio (RMSE)
0,0.1084,0.0196,0.1400


In [25]:
rede_neural.metricas('teste','regressao')

,Erro Médio Absoluto (MAE),Erro Quadrático Médio (MSE),Raiz do Erro Quadrático Médio (RMSE)
0,0.0946,0.0141,0.1187
